OpenAI´s function_Calling was first deprecated in favor of tool_calling - but now the correct terms seems to be function_calling again.
If you call it tool- or function calling - at the moment is does not really matter. 

Reference: https://platform.openai.com/docs/guides/function-calling

In [2]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [3]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content='I am unable to provide real-time weather updates. However, you can check the weather forecast for Munich today on various weather websites or apps such as Weather.com, AccuWeather, or the Weather Channel.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 17, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BuhZzrE7iPeZ2GzyzOmVuxKw2nlKl', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--31101b65-91c1-4170-9c58-486427666932-0', usage_metadata={'input_tokens': 17, 'output_tokens': 41, 'total_tokens': 58, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
from langchain_core.tools import tool


@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"


@tool
def outdoor_seating_availability(city: str) -> str:
    """
    Check if outdoor seating is available at a specified restaurant in a given city.

    Args:
        city (str): The name of the city where you want to check for outdoor seating availability.

    Returns:
        str: A message stating whether outdoor seating is available or not.
    """
    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]

In [ ]:
# from langchain_core.pydantic_v1 import BaseModel, Field

# class WeatherCheck(BaseModel):
#     """Check the weather in a specified city."""

#     city: str = Field(..., description="Name of the city to check the weather for")


# class OutdoorSeatingCheck(BaseModel):
#     """Check if outdoor seating is available at a specified restaurant in a given city."""

#     city: str = Field(..., description="Name of the city where the restaurant is located")


# tools = [WeatherCheck, OutdoorSeatingCheck]

In [6]:
llm_with_tools = llm.bind_tools(tools)

In [7]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h5bCUCrDRnr2qI6HI3wd22Hb', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 172, 'total_tokens': 189, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Buhci3SIbyOJ91t9Js5XBVbuW3tqp', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b0b95fcd-804f-422b-81a8-f2bc641f1ab4-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_h5bCUCrDRnr2qI6HI3wd22Hb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 172, 'output_tokens': 17, 'total_tokens': 189, 'input_token_details': {'audio

In [8]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nmNtNSIdnUzBTzDpAZn8DABx', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_LCuCA3VdxQd7FXed99ASWrtt', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 180, 'total_tokens': 232, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BuhdJ8lWa1urBDItZbrLwGOQUVtz8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8a73749d-c827-4bfc-b093-6e0e64cc2db2-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_nmNtNSId

In [9]:
result.tool_calls

[{'name': 'fake_weather_api',
  'args': {'city': 'Munich'},
  'id': 'call_nmNtNSIdnUzBTzDpAZn8DABx',
  'type': 'tool_call'},
 {'name': 'outdoor_seating_availability',
  'args': {'city': 'Munich'},
  'id': 'call_LCuCA3VdxQd7FXed99ASWrtt',
  'type': 'tool_call'}]

In [10]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? I would like to eat outside if possible"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [11]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ToSJyywo9MmsBcqFHVb7FMMV', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_c0PHntrohIy2n0ZGsS5JAw6R', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 180, 'total_tokens': 232, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Buhe3hTBy06hsNgaMUOwFW3qLBzel', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logpr

In [12]:
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

In [13]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [14]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ToSJyywo9MmsBcqFHVb7FMMV', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_c0PHntrohIy2n0ZGsS5JAw6R', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 180, 'total_tokens': 232, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Buhe3hTBy06hsNgaMUOwFW3qLBzel', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logpr

In [15]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is sunny with a temperature of 22°C. You can enjoy eating outside as outdoor seating is available.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 260, 'total_tokens': 287, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BuhexQAY32LZal05u3hqTd5DTQTVG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--be3b2d5a-1186-48a2-bf7f-454e53c41c1a-0', usage_metadata={'input_tokens': 260, 'output_tokens': 27, 'total_tokens': 287, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})